# **Lab 1**
### Modelling and computation in hazard assessment
### 26-27 Februrary 2024


In Lab 1, we introduce fundamental data analysis and modelling concepts that will be demostrated in the lectures of the Computation and Modelling in Hazard Assessment training.

*   In the first section, we will set the environment and show basic operations

*   Then we'll load a dataset with in vitro genotoxocity data and explore it by creating some illustrative visualisations

*   Finally, we'll develop our first neural network model to model genotoxicity using a classification model

Setup (run this block using the play button or Shift+Enter to install packages and download required data)

## Setup the environment

In [23]:
# import libraries
import numpy as np
import pandas as pd
import requests
from pathlib import Path

In [5]:
# mount the google drive in case we wish to persist results
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Basic operations

## Load and explore the genotoxicity data

In [25]:
# fetch the dataset from the github repository
import requests
from getpass import getpass

# Provide your GitHub credentials
token = getpass("Enter your GitHub token: ")

# Set up the file URL
file_url = "https://raw.githubusercontent.com/pkaramertzanis/modelling_hazard_assessment/master/datasets/genotoxicity_dataset.xlsx"  # Updated URL

# Use the token for authentication
headers = {'Authorization': f'token {token}'}
response = requests.get(file_url, headers=headers)

# Save the file locally
if response.status_code == 200:
    with open(Path(file_url).name, "wb") as file:
        file.write(response.content)
    print("File downloaded successfully!")
else:
    print(f"Failed to download file: {response.status_code}")

Enter your GitHub token: ··········
File downloaded successfully!


In [26]:
# load the dataset into a Pandas dataframe
gentox_data = pd.read_excel('genotoxicity_dataset.xlsx')
gentox_data.head()


,datapoint ID,smiles_std,genotoxicity,endpoint
0,0,B12B3B4B1C234,negative,"Salmonella typhimurium (TA 100), no"
1,1,B12B3B4B1C234,negative,"Salmonella typhimurium (TA 100), yes"
2,2,B12B3B4B1C234,negative,"Salmonella typhimurium (TA 1535), no"
3,3,B12B3B4B1C234,negative,"Salmonella typhimurium (TA 1535), yes"
4,4,B12B3B4B1C234,negative,"Salmonella typhimurium (TA 1537), no"
